In [1]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [2]:
import os
os.environ["ZHIPUAI_API_KEY"] = "6771943d3790217c139b3809aec6858f.np9HoDC4J6JIyyQh"

In [3]:
chat = ChatZhipuAI(
    model="glm-4-air",
    temperature=0.2,
)

In [4]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

name_schema = ResponseSchema(name="poi_name_list",
                             type="list[list[string]]",
                             description="以POI名称作为元素构成列表给出每一天行程的访问顺序")
cate_schema = ResponseSchema(name="poi_cate_list",
                             type="list[list[string]]",
                             description="以POI类型作为元素构成列表，对应到poi_name_list每一天的行程，具体类型有‘景点’‘餐厅’‘酒店’")
recommend_time_schema = ResponseSchema(name="poi_rec_time_list",
                             type="list[list[float]]",
                             description="以POI推荐游玩停留时间作为元素构成列表，对应到poi_name_list每一天的行程，以h为单位")

# 
# time_schema = ResponseSchema(name="time",
#                              description="以每日推荐时间作为元素构成的列表给出每一天行程的访问顺序")

response_schemas = [name_schema, cate_schema, recommend_time_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"poi_name_list": list[list[string]]  // 以POI名称作为元素构成列表给出每一天行程的访问顺序
	"poi_cate_list": list[list[string]]  // 以POI类型作为元素构成列表，对应到poi_name_list每一天的行程，具体类型有‘景点’‘餐厅’‘酒店’
	"poi_rec_time_list": list[list[float]]  // 以POI推荐游玩停留时间作为元素构成列表，对应到poi_name_list每一天的行程，以h为单位
}
```


In [5]:
messages = [
    AIMessage(content="你好，我是行程规划助手。"),
    SystemMessage(content=f"你是一个行程规划助手，你的工作是根据你的知识一次理解用户的需求（不可询问额外信息），根据单轮用户的需求给出合理的每日旅行计划，需要从酒店开始以酒店结束，每天需要至少包含2个餐厅和合适数量的景点，所有POI均需要给出完整准确的信息，不能缩写，输出不能包含注释。\n {format_instructions}"),
    HumanMessage(content="我想去天津游玩3天"),
]

In [6]:
response = chat.invoke(messages)
print(response)
print(response.content)

content='```json\n{\n\t"poi_name_list": [\n\t\t["天津之眼", "古文化街", "天津狗不理包子博物馆", "海河意式风情区", "酒店"],\n\t\t["瓷房子", "天津五大道", "张小泉美食街", "天津博物馆", "酒店"],\n\t\t["天津海昌极地海洋世界", "天津蓟州区的盘山风景名胜区", "耳朵眼炸糕店", "酒店"]\n\t],\n\t"poi_cate_list": [\n\t\t["景点", "景点", "餐厅", "景点", "酒店"],\n\t\t["景点", "景点", "餐厅", "景点", "酒店"],\n\t\t["景点", "景点", "餐厅", "酒店"]\n\t],\n\t"poi_rec_time_list": [\n\t\t[2.0, 2.0, 1.5, 2.0, 0.5],\n\t\t[2.0, 2.0, 1.5, 2.0, 0.5],\n\t\t[3.0, 3.0, 1.0, 0.5]\n\t]\n}\n```' response_metadata={'token_usage': {'completion_tokens': 247, 'prompt_tokens': 236, 'total_tokens': 483}, 'model_name': 'glm-4-air', 'finish_reason': 'stop'} id='run-005f1e09-b484-460a-9dbf-2bfda378e370-0'
```json
{
	"poi_name_list": [
		["天津之眼", "古文化街", "天津狗不理包子博物馆", "海河意式风情区", "酒店"],
		["瓷房子", "天津五大道", "张小泉美食街", "天津博物馆", "酒店"],
		["天津海昌极地海洋世界", "天津蓟州区的盘山风景名胜区", "耳朵眼炸糕店", "酒店"]
	],
	"poi_cate_list": [
		["景点", "景点", "餐厅", "景点", "酒店"],
		["景点", "景点", "餐厅", "景点", "酒店"],
		["景点", "景点", "餐厅", "酒店"]
	],
	"poi_rec_time_list": [
		[2.0,

In [7]:
output_dict = output_parser.parse(response.content)
output_dict

{'poi_name_list': [['天津之眼', '古文化街', '天津狗不理包子博物馆', '海河意式风情区', '酒店'],
  ['瓷房子', '天津五大道', '张小泉美食街', '天津博物馆', '酒店'],
  ['天津海昌极地海洋世界', '天津蓟州区的盘山风景名胜区', '耳朵眼炸糕店', '酒店']],
 'poi_cate_list': [['景点', '景点', '餐厅', '景点', '酒店'],
  ['景点', '景点', '餐厅', '景点', '酒店'],
  ['景点', '景点', '餐厅', '酒店']],
 'poi_rec_time_list': [[2.0, 2.0, 1.5, 2.0, 0.5],
  [2.0, 2.0, 1.5, 2.0, 0.5],
  [3.0, 3.0, 1.0, 0.5]]}

In [51]:
type(output_dict['poi_name_list'])

list